In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import Lasso, LinearRegression
from sklearn.cluster import KMeans

import plotly.express as px

import plotly.graph_objects as go

from scipy.linalg import toeplitz, block_diag

from spe.mse_estimator import ErrorComparer, gen_rbf_X, create_clus_split
from spe.relaxed_lasso import RelaxedLasso

In [2]:
err_cmp = ErrorComparer()

In [42]:
niter = 200

n=20**2
p=200
s=30

k=3
alpha=1.
lambd=.45

In [43]:
nx = ny = int(np.sqrt(n))
xs = np.linspace(0, 10, nx)
ys = np.linspace(0, 10, ny)
c_x, c_y = np.meshgrid(xs, ys)
c_x = c_x.flatten()
c_y = c_y.flatten()

In [44]:
rho = 0.5
delta = 0.8

D = np.abs(c_x - c_x[:,None]) + np.abs(c_y - c_y[:,None])
Sigma_t = rho**D
Sigma_t = delta*Sigma_t + (1-delta) * np.eye(n)

# m = 20
# c = np.zeros(n)
# c[:m] = [rho**i for i in np.arange(m)]
# Sigma_t = toeplitz(c)

# b = n / m
# c = [rho**i for i in np.arange(m)] 
# Sigma_t = block_diag(*[toeplitz(c) for _ in np.arange(b)])

Chol_t = np.linalg.cholesky(Sigma_t)
# Sigma_t = np.eye(n)
# Chol_t = np.eye(n)

In [45]:
# c_x = np.random.uniform(0,10,size=n)
# c_y = np.random.uniform(0,10,size=n)

# rn = int(np.sqrt(n))
# ctr_x = np.random.uniform(0,10,size=rn)
# ctr_y = np.random.uniform(0,10,size=rn)
# c_x = np.concatenate([c + .35 * np.random.randn(rn) for c in ctr_x])
# c_y = np.concatenate([c + .35 * np.random.randn(rn) for c in ctr_y])

X = gen_rbf_X(c_x, c_y, p)

beta = np.zeros(p)
idx = np.random.choice(p,size=s)
beta[idx] = np.random.uniform(-1,1,size=s)

In [46]:
# xv, yv = np.meshgrid(np.arange(nx), np.arange(ny))
# pts = np.stack([xv.ravel(), yv.ravel()]).T
# n = nx*ny
# rn = int(np.sqrt(n))
# ctr = np.random.choice(pts.shape[0], size=rn, replace=True)
# ctr = pts[ctr]
# tr_idx = np.vstack([[pt + np.array((1.25*np.random.randn(2)).astype(int)) for _ in np.arange(rn)] for pt in ctr])
# tr_idx = np.maximum(0, tr_idx)
# tr_idx[:,0] = cx = np.minimum(nx-1, tr_idx[:,0])
# tr_idx[:,1] = cy = np.minimum(ny-1, tr_idx[:,1])
# tr_idx = np.unique(np.ravel_multi_index(tr_idx.T, (nx,ny)))
# tr_idx.shape

In [47]:
(test_err,
 kfcv_err,
 spcv_err,
 rela_err) = err_cmp.compareRelaxedLassoTrTs(n=n,
                                     p=p,
                                     X=X,
                                     beta=beta,
                                     coord=np.stack([c_x, c_y]).T,
                                     Chol_t=Chol_t,
                                     niter=niter,
                                     alpha=alpha,
                                     lambd=lambd,
                                     k=k)
    

0
0.2325
10
zeros
20
30
40
zeros
50
60
zeros
70
80
90
100
zeros
110
120
130
140
zeros
150
160
zeros
170
zeros
zeros
180
190


In [48]:
# groups = KMeans(n_clusters=10).fit(tr_idx).labels_

In [49]:
# px.scatter(pd.DataFrame({'x': tr_idx[:,0], 'y': tr_idx[:,1], 'g': groups}), x='x', y='y', color='g')

In [50]:
risk = test_err.mean()
risk_kfcv = kfcv_err.mean()
risk_spcv = spcv_err.mean()
risk_rela = rela_err.mean()
risk, risk_kfcv, risk_spcv, risk_rela

(2.098292920124134, 1.5913222419082045, 2.5178154642103463, 2.1445973417993915)

In [51]:
df = pd.DataFrame({'GenCp': (rela_err.T),# - test_err.T) / test_err.T,
                   'KFCV': (kfcv_err.T),# - test_err.T) / test_err.T,
                   'SPCV': (spcv_err.T)})# - test_err.T) / test_err.T})

In [52]:
(pd.DataFrame({'GenCp': (rela_err.T),
                   'KFCV': (kfcv_err.T),
                   'SPCV': (spcv_err.T)}) /risk).to_csv("~/Downloads/rl_df.csv")

In [53]:
fig_rela = px.box((df - risk) / risk, 
             labels={
                     "variable": "Method",
                     "value": "Relative MSE"
                     },
             title="lin correction",
             points=False)
fig_rela.update_traces(boxmean=True)

fig_rela.add_hline(y=0., line_color='red')

In [54]:
fig = go.Figure()
fig.add_trace(go.Bar(
    name='Control',
    x=['GenCp', 'KFCV', 'SPCV'], y=(df/risk).mean(),
    marker_color=px.colors.qualitative.Plotly,
    text=np.around((df/risk).mean(),3),
    textposition='outside',
    error_y=dict(
        type='data',
        color='black',
        symmetric=False,
        array=(df/risk).quantile(.75),
        arrayminus=(df/risk).quantile(.25))
))
# fig.add_trace(go.Bar(
#     name='Experimental',
#     x=['GenCp', 'KFCV', 'SPCV'], y=(df/risk).mean(),
#     error_y=dict(type='data', array=[1, 2])
# ))
# fig.update_layout(barmode='group')
fig.add_hline(y=1., line_color='red')
fig.update_layout(
    title="Relaxed Laso Test Error Estimates",
    xaxis_title="Method",
    yaxis_title="Relative MSE",
#     legend_title="Legend Title",
#     font=dict(
#         family="Courier New, monospace",
#         size=18,
#         color="RebeccaPurple"
#     )
)
# ticktext= []
# for i in np.arange()
# fig.update_layout(
#     yaxis = dict(
#         tickmode = 'linear',
#         tick0 = 0.,
#         dtick = 1.,
# #         ticktext = ticktext,
#     )
# )
fig.show()